# Agent Notebook
This notebook demonstrates an agent workflow using autogen and models hosted on Ollama.

In [44]:
import importlib.util, subprocess, sys

packages = {
    'autogen': 'autogen',
    'ag2[openai]': 'ag2',
    'autogen-ext[ollama]': 'autogen.ext',
    'jupytext': 'jupytext',
    'markdownify': 'markdownify',
    'accelerate': 'accelerate',
    'bitsandbytes': 'bitsandbytes',
    'requests': 'requests',
}

for pkg, module in packages.items():
    if importlib.util.find_spec(module) is None:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-U', pkg])


## Imports and constants

In [45]:
from autogen import AssistantAgent, UserProxyAgent, GroupChat, GroupChatManager

#from autogen import OpenAIChatCompletionClient as _OAIClient
import openai
#OpenAIChatCompletionClient = _OAIClient

import os, pathlib, datetime, uuid, subprocess, shlex, markdownify, sys, io, contextlib

REPO_ROOT = pathlib.Path.cwd()
LOG_DIR = REPO_ROOT / 'agent_logs'
LOG_DIR.mkdir(exist_ok=True, parents=True)
BASE_URL = os.environ.get('OLLAMA_BASE_URL', 'http://docker-ai:11434/v1')


## Test Ollama Server

In [46]:
import requests
def ensure_ollama():
    url=f'{BASE_URL}/models'
    try:
        resp=requests.get(url, timeout=5)
        resp.raise_for_status()
        data=resp.json()
        names=[m.get('id') or m.get('name') for m in data.get('data', data.get('models', []))]
        print('Ollama models:', ', '.join(names))
    except Exception as e:
        raise RuntimeError(f'Ollama server not reachable at {BASE_URL}: {e}')

ensure_ollama()


Ollama models: Qwen2.5-14B-Instruct-1M-Q8_0:latest, gguf/DeepSeek-Janus-Pro-7B:Q8_0, llama3.2-vision:11b, Qwen3-32B-Q5_0:latest, deepseek-ai_DeepSeek-R1-0528-Qwen3-8B-Q8_0:latest, Llama-3.2-3B-Instruct-Q8_0:latest, qwq:32b, qwen3:32b, devstral:24b, deepseek-r1:8b, gemma3:27b


## Model clients

In [47]:
qwen14_client = OpenAIChatCompletionClient(
    model='Qwen2.5-14B-Instruct-1M-Q8_0:latest',
    base_url=BASE_URL,
    api_key='ollama')

qwen32_client = OpenAIChatCompletionClient(
    model='Qwen3-32B-Q5_0:latest',
    base_url=BASE_URL,
    api_key='ollama')

devstral_client = OpenAIChatCompletionClient(
    model='devstral:24b',
    base_url=BASE_URL,
    api_key='ollama')


## Markdown logger

In [48]:
def log_markdown(log_dir: pathlib.Path, role, content, print_stdout=True):
    ts = datetime.datetime.utcnow().isoformat()
    fn = log_dir / 'log.md'
    if not fn.exists():
        log_dir.mkdir(parents=True, exist_ok=True)
        with open(fn, 'w') as f:
            f.write(f'---\nid: {log_dir.name}\ncreated: {ts}\n---\n\n')
    with open(fn, 'a') as f:
        f.write(f'### {ts} — {role}\n\n{markdownify.markdownify(content)}\n\n')
    if print_stdout:
        print(f'[{ts}] {role}: {content}')


## Shell helper

In [49]:
def run_shell(cmd: str) -> str:
    out = subprocess.check_output(shlex.split(cmd), text=True, timeout=900, stderr=subprocess.STDOUT)
    return f'```shell\n$ {cmd}\n{out}\n```'


## Agent declarations

In [50]:
planner = AssistantAgent(
    name='planner',
    llm_config={'config_list': [{ 'model': qwen14_client.model, 'base_url': qwen14_client.base_url, 'api_key': qwen14_client.api_key }], 'temperature': 0.3},
    system_message=("You are a project planner. Break the user's request into a YAML list of atomic tasks. Stop when each sub-task can be executed in one short Python call or shell command inside the current Jupyter kernel."),
)

worker = AssistantAgent(
    name='worker',
    llm_config={'config_list': [{ 'model': qwen32_client.model, 'base_url': qwen32_client.base_url, 'api_key': qwen32_client.api_key }], 'temperature': 0},
    system_message='Execute the given atomic task and return result.')
worker.register_function({'run_shell': run_shell})

coder = AssistantAgent(
    name='coder',
    llm_config={'config_list': [{ 'model': devstral_client.model, 'base_url': devstral_client.base_url, 'api_key': devstral_client.api_key }], 'temperature': 0},
    system_message='You are a senior software engineer. Write, refactor, and debug code snippets as requested.')

reviewer = AssistantAgent(
    name='reviewer',
    llm_config={'config_list': [{ 'model': qwen14_client.model, 'base_url': qwen14_client.base_url, 'api_key': qwen14_client.api_key }], 'temperature': 0},
    system_message=("Evaluate the worker or coder output against the task description. If incorrect, respond with REVISE and instructions; otherwise APPROVED."),
)

agents = [planner, worker, coder, reviewer]
group = GroupChat(agents=agents, max_round=30)
manager = GroupChatManager(groupchat=group, llm_config={'config_list': [{ 'model': qwen14_client.model, 'base_url': qwen14_client.base_url, 'api_key': qwen14_client.api_key }], 'temperature': 0})
proxy = UserProxyAgent(name='user', human_input_mode='NEVER', code_execution_config={'use_docker': False})


## Driver function

In [51]:
def run_agent(prompt: str):
    run_dir = LOG_DIR / datetime.datetime.utcnow().strftime('%Y-%m-%d-%H-%M-%S-%f')
    log_markdown(run_dir, 'INFO', 'Agent run started')
    log_markdown(run_dir, 'USER', prompt)
    class LogStream(io.TextIOBase):
        def __init__(self):
            self._stdout = sys.__stdout__
        def write(self, s):
            if s:
                self._stdout.write(s)
                if s.strip():
                    log_markdown(run_dir, 'VERBOSE', s, print_stdout=False)
            return len(s)
        def flush(self):
            self._stdout.flush()
    stream = LogStream()
    with contextlib.redirect_stdout(stream), contextlib.redirect_stderr(stream):
        try:
            proxy.initiate_chat(manager, message=prompt)
        except Exception as e:
            log_markdown(run_dir, 'ERROR', f'LLM request failed: {e}')
            raise
    for m in group.chat_history:
        log_markdown(run_dir, m['role'], m['content'])
    log_markdown(run_dir, 'INFO', 'Agent run complete')
    return run_dir / 'log.md'


## Example call

In [52]:
#run_agent('Generate Python code to scrape example.com daily and store results in SQLite …')
run_agent('Analyze all the files in the papers directory recursively. For each paper, analyze this notebook and make recommendations based on the arguments made in the paper Save each in a separate file within the agent_logs directory for this run.')

C:\Users\CJ\AppData\Local\Temp\ipykernel_3856\400404312.py:2: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  run_dir = LOG_DIR / datetime.datetime.utcnow().strftime('%Y-%m-%d-%H-%M-%S-%f')


[2025-07-07T05:51:58.268411] INFO: Agent run started
[2025-07-07T05:51:58.269322] USER: Generate Python code to scrape example.com daily and store results in SQLite …


## Git auto-commit

In [ ]:
!git add agent_logs/** && (git diff --cached --quiet || git commit -m 'agent run') && git push

## Version info

In [ ]:
!pip freeze | grep -E '(autogen|transformers|ollama)'